In [2]:
import hashlib
from ecdsa import SECP256k1, SigningKey
import sys
import binascii

# 58 character alphabet used
BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'

def from_bytes (data, big_endian = False):
    if isinstance(data, str):
        data = bytearray(data)
    if big_endian:
        data = reversed(data)
    num = 0
    for offset, byte in enumerate(data):
        num += byte << (offset * 8)
    return num
    
def base58_encode(version, public_address):
    """
    Gets a Base58Check string
    See https://en.bitcoin.it/wiki/Base58Check_encoding
    """
    if sys.version_info.major > 2:
        version = bytes.fromhex(version)
    else:
        version = bytearray.fromhex(version)
    firstSHA256 = hashlib.sha256(version + public_address)
    print("first sha256: %s"%firstSHA256.hexdigest().upper())
    secondSHA256 = hashlib.sha256(firstSHA256.digest())
    print("second sha256: %s"%secondSHA256.hexdigest().upper())
    checksum = secondSHA256.digest()[:4]
    payload = version + public_address + checksum
    print("Hex address: %s"%binascii.hexlify(payload).decode().upper())
    if sys.version_info.major > 2:
        result = int.from_bytes(payload, byteorder="big")
    else:
        result = from_bytes(payload, True)
    # count the leading 0s
    padding = len(payload) - len(payload.lstrip(b'\0'))
    encoded = []

    while result != 0:
        result, remainder = divmod(result, 58)
        encoded.append(BASE58_ALPHABET[remainder])

    return padding*"1" + "".join(encoded)[::-1]

def get_private_key(hex_string):
    if sys.version_info.major > 2:
        return bytes.fromhex(hex_string.zfill(64))
    else:
        return bytearray.fromhex(hex_string.zfill(64))

def get_public_key(private_key):
    # this returns the concatenated x and y coordinates for the supplied private address
    # the prepended 04 is used to signify that it's uncompressed
    if sys.version_info.major > 2:
        return (bytes.fromhex("04") + SigningKey.from_string(private_key, curve=SECP256k1).verifying_key.to_string())
    else:
        return (bytearray.fromhex("04") + SigningKey.from_string(private_key, curve=SECP256k1).verifying_key.to_string())

def get_public_address(public_key):
    address = hashlib.sha256(public_key).digest()
    print("public key hash256: %s"%hashlib.sha256(public_key).hexdigest().upper())
    h = hashlib.new('ripemd160')
    h.update(address)
    address = h.digest()
    print("RIPEMD-160: %s"%h.hexdigest().upper())
    print(len(h.hexdigest().upper()))
    return address

if __name__ == "__main__":
    #private_key = get_private_key("FEEDB0BDEADBEEF")
    private_key = get_private_key("18e14a7b6a307f426a94f8114701e7c8e774e7f9a47e2c2035db29a206321726")
    print("private key: %s"%binascii.hexlify(private_key).decode().upper())
    public_key = get_public_key(private_key)
    print("public_key: %s"%binascii.hexlify(public_key).decode().upper())
    public_address = get_public_address(public_key)
    bitcoin_address = base58_encode("00", public_address)
    print("Final address %s"%bitcoin_address)

# PubKeyToAddress将比特币公钥转换为base58编码的字符串地址
# 在PubKeyToAddress函数中，会先调用Hash160对公钥进行SHA256和RIPEMD160哈希运算
# 即对应生成过程的第三、四步

private key: 18E14A7B6A307F426A94F8114701E7C8E774E7F9A47E2C2035DB29A206321726
public_key: 046EAF0968AA895ADDFEE599566F0B880242461D1377F4887C9B84631E13067B96DB18C41E0C208F8D12EBCC3F99F2522903AF6105833E4CBADE9D6A1D0F039187
public key hash256: 1DC4B1F3A6552A05A4F308FF03E412D699BE937DC7CFAF5BD27AA01495593B0E
RIPEMD-160: 8915DD5B4A59FEC43981431E106380EA8AFA22BC
40
first sha256: 3EB136652F264B0F11E6480D559815EC7457B1DCC4038A23262E24A8E051856E
second sha256: 4BE62CB707D5082293EF3F425B26A6563DBCE83F10AB5889D96BC69454575E9C
Hex address: 008915DD5B4A59FEC43981431E106380EA8AFA22BC4BE62CB7
Final address 1DVqmTy9Ux3NqkWURZZfygqvnxac2FjH66


private key: 18E14A7B6A307F426A94F8114701E7C8E774E7F9A47E2C2035DB29A206321725
public_key: 0450863AD64A87AE8A2FE83C1AF1A8403CB53F53E486D8511DAD8A04887E5B23522CD470243453A299FA9E77237716103ABC11A1DF38855ED6F2EE187E9C582BA6
public key hash256: 600FFE422B4E00731A59557A5CCA46CC183944191006324A447BDB2D98D4B408
RIPEMD-160: 010966776006953D5567439E5E39F86A0D273BEE
40
first sha256: 445C7A8007A93D8733188288BB320A8FE2DEBD2AE1B47F0F50BC10BAE845C094
second sha256: D61967F63C7DD183914A4AE452C9F6AD5D462CE3D277798075B107615C1A8A30
Hex address: 00010966776006953D5567439E5E39F86A0D273BEED61967F6
Final address 16UwLL9Risc3QfPqBUvKofHmBQ7wMtjvM

In [1]:
# BASE58解码
# 第一步 反解 Base58，生成16进制的地址 和 4 位校验码
BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
xx = input("请输bitcoin地址:")
# print(xx)
num_list = []
for x in xx:
    num_list.append(BASE58_ALPHABET.index(x))
# print(num_list)
num = 0
for i in range(len(num_list)):
    num += num_list[i] * (58 ** (len(num_list)-1-i))

addr_16 = (hex(num).upper())
print("16进制地址为:",addr_16)
#  生成的4位校验码
check_4num = addr_16[-8:]
print("四位校验码为:",check_4num)
after_ripemd160 = addr_16[2:-8]
#print(len(after_ripemd160))
# 第二步 得到RIPEMD-16编码后的16进制数
while len(after_ripemd160) < 40:
    after_ripemd160 = "0" + after_ripemd160
print("RIPEMD-160编码后的16进制数",after_ripemd160)
# 第三步 反解RIPEMD-160得到公钥的SHA-256值
# 第四步 反解SHA-256得到公钥共65字节，前面子字节为04，X坐标32字节，Y坐标32字节
# 第五步 反解椭圆曲线算法 得到一个32字节的16进制数(64位) 即为私钥！！！

请输bitcoin地址:16UwLL9Risc3QfPqBUvKofHmBQ7wMtjvN
16进制地址为: 0X10966776006953D5567439E5E39F86A0D273BEED61967F7
四位校验码为: D61967F7
RIPEMD-160编码后的16进制数 010966776006953D5567439E5E39F86A0D273BEE


In [ ]:
# BASE58编码

In [ ]:
x = "0"
hex_num = ""
for i in range(64):
    hex_num += x
# hex_num = "8888888888888888888888888888888888888888888888888888888888888888"
print(hex_num)

In [366]:
import random
max_bin256 = 0b1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111010111010101011101101110011100110101011110100100010100000001110111011111111010010010111101000110011010000001101100100000101000001
# print(bin(random.randint(0b0,max_bin256)))
rand_bin = bin(random.randint(0b0,max_bin256))[2:]
# print(len(rand_bin))
sum_bin = 0
for i in rand_bin:
    sum_bin += int(i)
print(sum_bin)

255
137
